In [151]:
# %pip install sqlalchemy En caso de ser necesario, ejecutar
# %pip install --user psycopg2 En caso de ser necesario, ejecutar
# %pip install numpy En caso de ser necesario, ejecutar
# %pip install pandas En caso de ser necesario, ejecutar

In [152]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [153]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlFuente = f'{configFuente['driver']}://{configFuente['user']}:{configFuente['password']}@{configFuente['host']}:{configFuente['port']}/{configFuente['db']}'
urlBodega = f'{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}'

src = create_engine(urlFuente)
etl = create_engine(urlBodega)

In [154]:
novedad_servicio = pd.read_sql_table('mensajeria_novedadesservicio', src)
mensajero = pd.read_sql_table('dim_mensajero', etl)
fecha = pd.read_sql_table('dim_fecha', etl)

# Seleccionar solo las columnas 'id', 'mensajero_id' y 'fecha_novedad'
hecho_novedad1 = novedad_servicio[['id', 'mensajero_id', 'fecha_novedad', 'servicio_id']].copy()

# Renombrar la columna 'servicio_id' a 'id_servicio'
hecho_novedad1.rename(columns={'servicio_id': 'id_servicio'}, inplace=True)


# Asegúrate de que tienes la columna común entre 'hecho_novedad' y 'mensajero'
hecho_novedad1 = hecho_novedad1.merge(mensajero[['key_dim_mensajero']], left_on='mensajero_id', right_on='key_dim_mensajero', how='left')

hecho_novedad1.drop(columns=['mensajero_id'], inplace=True)

# Eliminar la hora de 'fecha_novedad' y dejar solo la fecha (año, mes, día)
hecho_novedad1['fecha_novedad'] = hecho_novedad1['fecha_novedad'].dt.date


# Ahora ambas columnas tienen el mismo formato
# Asegurarte de que ambas columnas son de tipo datetime64
hecho_novedad1['fecha_novedad'] = pd.to_datetime(hecho_novedad1['fecha_novedad'])
fecha['fecha'] = pd.to_datetime(fecha['fecha'])

# Realizar el merge ahora
hecho_novedad1 = hecho_novedad1.merge(fecha[['key_dim_fecha', 'fecha']], left_on='fecha_novedad', right_on='fecha', how='left')
hecho_novedad1.drop(columns=['id','fecha_novedad','fecha'], inplace=True)



hecho_novedad2 = novedad_servicio.groupby(['servicio_id', 'tipo_novedad_id']).size().reset_index(name='total_novedad')

hecho_novedad2.rename(columns={
    'servicio_id': 'key_servicio',
    'tipo_novedad_id': 'key_tipo_novedad',
}, inplace=True)

hecho_novedad2["key_hecho_novedad"] = range(1, len(hecho_novedad2) + 1)

hecho_novedad = hecho_novedad1.merge(hecho_novedad2, left_on='id_servicio', right_on='key_servicio', how='left')
hecho_novedad.drop(columns=['id_servicio'], inplace=True)



# Mover la columna 'telefono' al final
cols = [col for col in hecho_novedad.columns if col != 'total_novedad'] + ['total_novedad']
hecho_novedad = hecho_novedad[cols]


# Eliminar filas duplicadas en 'hecho_novedad'
hecho_novedad = hecho_novedad.drop_duplicates()


# Ahora el DataFrame 'hecho_novedad' tiene la columna 'key_dim_mensajero'
hecho_novedad




,key_dim_mensajero,key_dim_fecha,key_servicio,key_tipo_novedad,key_hecho_novedad,total_novedad
0,7,334,51,1,3,5
1,7,334,51,2,4,3
16,7,334,29,1,1,1
17,7,334,29,2,2,1
18,7,341,29,1,1,1
...,...,...,...,...,...,...
5969,25,609,28461,2,3559,1
5970,27,609,28455,1,3558,1
5971,25,609,28464,2,3560,1
5972,25,609,28467,2,3562,2


In [155]:
hecho_novedad.to_sql('hecho_novedad', etl, index=False, if_exists='replace')

875